In [1]:
!pip install transformers trl accelerate torch bitsandbytes peft datasets huggingface_hub -qU

In [5]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
!pip install -U peft

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
from peft import PeftConfig, PeftModel

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True, #load weights in 4 bits
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16 #update bits in 16 bits
)

base_model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.1",
    device_map='auto',
    quantization_config=nf4_config,
    trust_remote_code=True,
    use_cache=False
)

PEFT_MODEL = "akashvshroff/mistral-7b-math"

config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=nf4_config,
    trust_remote_code=True
)

model = PeftModel.from_pretrained(model, PEFT_MODEL)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/27.3M [00:00<?, ?B/s]

In [8]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [9]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0].replace(prompt, "")

In [25]:
prompt = "I work for $12 an hour and yesterday I worked for 3 and a half hours. How much did I earn? Explain your answer."

In [26]:
generate_response(prompt, model)

"<s> \n\nLet's calculate this step by step:\n\n1. First, let's find out how much you earn per hour: $12/hour.\n2. Next, we need to figure out how much you earned for the 3 and a half hours you worked: \n   Earnings = Hourly rate x Hours worked\n   = $12/hour x 3.5 hours\n   = $42\n\nSo, you earned $42 for working 3 and a half hours.</s>"

In [27]:
generate_response(prompt, base_model)

'<s> \n\nTo answer this problem, we first need to figure out how many hours the person worked. We are given that they worked 3.5 hours.\nNext step is to multiply the length of time worked by the amount of money earned per hour. In this case, that amount is 12 dollars an hour. So, if the person worked 3.5 hours and earned 12 dollars an hour, then they made 38 dollars.\nSo, the person worked for 3.5 hours and earned 12 dollars an hour for each hour that they worked, for a total of 38 dollars.\nIf you look above, I put all the steps of the math in an equation so you can understand it better.\nWork, time.=Money earned, per hour,per time.=Total money earned.\n=38\n=3+1=4\n=8\nMoney earned per hour and worked for x hour.=Total earned.\n=38\n=3+1=4\n=8\nWork=1\nTime=3.5\nMoney earned per hour.Time= 8/3.5=2\n=4\n=4\n=8/3.5=2\n=4\n=8/3.5=2\n=3.5\nAnswer: 12 (Money Per Hour) multiplied by (3.5 (Hours worked)) equals (38 (Total Money Earned))\n=4\n=4\n=8\n=3\n=12\n=0.8\n=8/3.5=2\n=0.8\n=2\n=3.5\n